In [ ]:
import csv
import cv2
import datetime
import ipynbname
import math
import numpy
import os
import pandas
import pathlib
import skimage.io
import skimage.transform
import sklearn.model_selection
import tensorflow
import time

from albumentations import (
    Compose, HorizontalFlip, ShiftScaleRotate, ElasticTransform,
    RandomBrightness, RandomContrast, RandomGamma
)

from metrics import dice_coef, jaccard_distance
from model import unet_model, get_loss_function
from AugmentationSequence import AugmentationSequence

# GPU

In [ ]:
gpus = tensorflow.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            print(f"GPU: {gpu.name}")
            tensorflow.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
cfg = {
    "channel": 3,
    "batch_size": 4,
    "fold": 5,
    "epochs": 75,
    "image_size": 256,
    "learning_rate": 0.001,
    "random_state": 1234,
    "test_size": 0.2,
    "val_size": 0.05,
    "path_dataset": "dataset",
    "path_out": "out",
    "loss_function": "dice",
    "data_augmentation": True
}
images_folder = os.path.join(cfg["path_dataset"], "IMAGEM_ORIGINAL", "CONVERTIDAS", "RGB", "256", "OUT")
masks_folder = os.path.join(cfg["path_dataset"], "MASK", "BITMAP", "256", "OUT")

In [ ]:
list_labels = list([])
list_images = list([])
list_images_names = list([])
def load_files():
    for file in sorted(pathlib.Path(masks_folder).rglob("*")):
        mask = skimage.io.imread(file.resolve())
        mask = numpy.float32(mask/255)
        list_labels.append(mask)

        image = skimage.io.imread(os.path.join(images_folder, f"{file.stem}.png"))
        image = numpy.float32(image/255)
        list_images.append(image)

        list_images_names.append(str(file.stem))

load_files()
print(len(list_labels), len(list_images), len(list_images_names))

In [ ]:
x = numpy.array(list_images).reshape((len(list_images), cfg["image_size"], cfg["image_size"], cfg["channel"]))
y = numpy.array(list_labels).reshape((len(list_labels), cfg["image_size"], cfg["image_size"], 1))

x_train, x_val, y_train, y_val = sklearn.model_selection.train_test_split(x, y, test_size=cfg["val_size"], random_state=cfg["random_state"])

print(x.shape, y.shape)

In [ ]:
current_datetime = datetime.datetime.now().strftime("%d-%m-%Y-%H-%M-%S")
path_model = os.path.join(cfg["path_out"], "train", current_datetime)
pathlib.Path(path_model).mkdir(parents=True, exist_ok=True)

augment = Compose([
    HorizontalFlip(),
    ShiftScaleRotate(rotate_limit=45, border_mode=cv2.BORDER_CONSTANT),
    ElasticTransform(border_mode=cv2.BORDER_CONSTANT),
    RandomBrightness(),
    RandomContrast(),
    RandomGamma()
])
steps_per_epoch = math.ceil(x_train.shape[0] / cfg["batch_size"])
train_generator = AugmentationSequence(x_train, y_train, cfg["batch_size"], augment)
reduce_learning_rate = tensorflow.keras.callbacks.ReduceLROnPlateau(monitor="loss", factor=0.5, patience=3, verbose=1)
filename_model = os.path.join(path_model, "unet.h5")
checkpointer = tensorflow.keras.callbacks.ModelCheckpoint(filename_model, verbose=1, save_best_only=True)
strategy = tensorflow.distribute.MirroredStrategy()

with strategy.scope():
    model = unet_model(cfg)
    adam_opt = tensorflow.keras.optimizers.Adam(learning_rate=cfg["learning_rate"])
    model.compile(optimizer=adam_opt, loss=get_loss_function(cfg["loss_function"]), metrics=[dice_coef, jaccard_distance, tensorflow.keras.metrics.Precision(), tensorflow.keras.metrics.Recall()])

tensorflow.keras.backend.clear_session()
start_time = time.time()
fit = model.fit(train_generator, steps_per_epoch=steps_per_epoch, epochs=cfg["epochs"], validation_data=(x_val, y_val), callbacks=[checkpointer, reduce_learning_rate])
end_time = time.time() - start_time

In [ ]:
columns = ["batch_size", "epochs", "learning_rate", "loss_function", "images", "masks", "len_images", "len_masks", "channel", "image_size", "fold", "test_size", "val_size", "random_state", "path_dataset", "path_out", "data_augmentation", "filename_script", "time"]
data = [cfg["batch_size"], cfg["epochs"], cfg["learning_rate"], cfg["loss_function"], images_folder, masks_folder, len(list_images), len(list_labels), cfg["channel"], cfg["image_size"], cfg["fold"], cfg["test_size"], cfg["val_size"], cfg["random_state"], cfg["path_dataset"], cfg["path_out"], cfg["data_augmentation"], ipynbname.name(), time.strftime("%H:%M:%S", time.gmtime(end_time))]

dataframe_cfg = pandas.DataFrame(data, columns)
dataframe_cfg = dataframe_cfg.applymap(lambda x: str(x).replace(".", ",") if isinstance(x,float) else x)
dataframe_cfg.to_csv(os.path.join(path_model, "cfg.csv"), decimal=",", sep=";", na_rep=" ", header=False, quoting=csv.QUOTE_ALL)
dataframe_cfg